In [ ]:
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from scipy import spatial
from statistics import mode
import sklearn.metrics.pairwise
from statistics import mean
import random
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

In [ ]:
(train_x,train_y),(test_x, test_y) = mnist.load_data()
new_train = []
for i in train_x[0:4000]:
  new_train.append(i.flatten())
sample_train = new_train[0:]

#Purity must be more than .5

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
#K means algorithm
k = 10
cluster_assignments = defaultdict(list)
cluster_idx = defaultdict(list)

#Initialization of clusters
cluster_index = random.sample(range(0,4000),k)
cluster_list = [new_train[i] for i in cluster_index]

#Distance matrix
Euc_result = sklearn.metrics.pairwise.euclidean_distances(sample_train,cluster_list)
for i in range(len(sample_train)):
  cluster_num = np.argsort(Euc_result[i])[0]
  cluster_assignments[cluster_num].append(sample_train[i])


iter = 0
while(iter < 200):

#M Step
  new_assignments = defaultdict(list)
  new_cluster_list = np.array([])

  for i in range(k):
    new_cluster = np.average(cluster_assignments[i], axis = 0)
    new_cluster_list = np.append(new_cluster_list, new_cluster)
       
  new_cluster_list = np.reshape(new_cluster_list,(-1,784))

  
  if np.array_equal(new_cluster_list,cluster_list):
    print('Entering if statement')
    break

  cluster_list = new_cluster_list
  cluster_assignments = defaultdict(list)
  cluster_idx = defaultdict(list)


#E step
  Euc_result = sklearn.metrics.pairwise.euclidean_distances(sample_train,cluster_list)
  for i in range(len(sample_train)):
    cluster_num = np.argsort(Euc_result[i])[0]
    cluster_assignments[cluster_num].append(sample_train[i])
    cluster_idx[cluster_num].append(i)
  iter +=1
#
print('num of iterations ', iter)
purity = []
counted = []
count = 0
for i in range(len(cluster_idx)):
  labels = []
  for x in cluster_idx[i]:
    label = train_y[x]
    labels.append(label)
  #print(len(labels))
  label_mode = mode(labels)
  purity.append(labels.count(label_mode)/len(labels))
purity_num = sum(purity)/k
print('Average Purity is ', purity_num)

# #Gini Index
#Per cluster first calculate Gini index =  1- sum((num of points in a label/total points in cluster)^2)
gini_index = {}
for i in range(len(cluster_idx)):
  labels = []
  label_dict = {}
  for x in cluster_idx[i]:
    label = train_y[x]
    labels.append(label)

    if label not in label_dict:
      label_dict[label] = 1
    else:
      label_dict[label] += 1
  label_counts = list(label_dict.values())
  gin_sum = 0
  for y in range(len(label_dict)):
    gin_sum += (label_counts[y]/len(labels))*(label_counts[y]/len(labels))
  gini_index[i] = 1-gin_sum
#Next do the weighted avg (Gini index*num points in cluster/total points and add)
print(gini_index)
gini_avg = 0
for i in range(len(cluster_idx)):
  gini_avg += (gini_index[i]*len(cluster_idx[i])/len(sample_train))
print('Avg Gini Index is', gini_avg)


Entering if statement
num of iterations  38
Average Purity is  0.6374017697444745
{0: 0.7002921282227232, 1: 0.4968913546188486, 2: 0.5827335640138409, 3: 0.18241813321452427, 4: 0.10004756242568369, 5: 0.7064512700605804, 6: 0.7445351473922903, 7: 0.15678953626634962, 8: 0.6928061224489797, 9: 0.4986771060028298}
Avg Gini Index is 0.5257538629972143
